# Download Data

Download all data provided by the organizers.
There are multiple data repositories, according to a nomenclature defined here https://github.com/ecmwf-lab/climetlab-s2s-ai-challenge.



In [12]:
import bs4
import multiprocessing
import os
import pathlib
import random
import requests
import urllib

from tqdm.notebook import tqdm

INDEX_TRAIN_INPUT = 'https://storage.ecmwf.europeanweather.cloud/s2s-ai-challenge/data/training-input/0.3.0/netcdf/index.html'
INDEX_TRAIN_OUTPUT = 'https://storage.ecmwf.europeanweather.cloud/s2s-ai-challenge/data/training-output-benchmark/index.html'
INDEX_TRAIN_REFERENCE = 'https://storage.ecmwf.europeanweather.cloud/s2s-ai-challenge/data/training-output-reference/index.html'

INDEX_TEST_INPUT = 'https://storage.ecmwf.europeanweather.cloud/s2s-ai-challenge/data/test-input/0.3.0/netcdf'
INDEX_TEST_OUTPUT = 'https://storage.ecmwf.europeanweather.cloud/s2s-ai-challenge/data/test-output-benchmark/index.html'
INDEX_TEST_REFERENCE = 'https://storage.ecmwf.europeanweather.cloud/s2s-ai-challenge/data/test-output-reference/index.html'
HOME = "C:\\Users\\klow55\\github\\crims2s"
TARGET_DIR = os.path.join(HOME,'hdd_scratch\\s2s')
#os.makedirs(TARGET_DIR)

In [13]:
def read_index(index_url):
    html = requests.get(index_url).text
    soup = bs4.BeautifulSoup(html, 'html.parser')
    
    table = soup.find_all('tbody')[0]
    links = table.find_all('a')
    dataset_files = [a_tag.attrs['href'] for a_tag in links]
    
    return dataset_files

In [16]:
index_url=INDEX_TRAIN_REFERENCE
html = requests.get(index_url).text
html

'<?xml version="1.0" encoding="UTF-8"?><Error><Code>AccessDenied</Code><BucketName>s2s-ai-challenge</BucketName><RequestId>tx0000000000000152ef050-0062e48819-25c44be5-default</RequestId><HostId>25c44be5-default-default</HostId></Error>'

In [14]:
def download_one_dataset_file(file_url):
    """Inspired by https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests."""
    url_path = urllib.parse.urlparse(file_url).path
    download_path = pathlib.Path(TARGET_DIR) / pathlib.Path(url_path[23:])
    download_path.parent.mkdir(parents=True, exist_ok=True)
    
    with requests.get(file_url, stream=True) as stream:
        if os.path.isfile(download_path):
            """Ignore file if it already exists and file size is ok."""
            stream_len = int(stream.headers['Content-length'])
            local_len = os.path.getsize(download_path)

            if stream_len == local_len:               
                return download_path
        
        
        with open(download_path, 'wb') as f:
            for chunk in stream.iter_content(chunk_size=8192):
                f.write(chunk)
                
    return download_path

In [15]:
dataset_files = []
for index in [INDEX_TRAIN_OUTPUT, INDEX_TRAIN_INPUT, INDEX_TRAIN_REFERENCE, INDEX_TEST_REFERENCE, INDEX_TEST_INPUT, INDEX_TEST_OUTPUT]:
    print(index)
    dataset_files.extend(read_index(index))

https://storage.ecmwf.europeanweather.cloud/s2s-ai-challenge/data/training-output-benchmark/index.html
https://storage.ecmwf.europeanweather.cloud/s2s-ai-challenge/data/training-input/0.3.0/netcdf/index.html
https://storage.ecmwf.europeanweather.cloud/s2s-ai-challenge/data/training-output-reference/index.html


IndexError: list index out of range

In [ ]:
local_file = download_one_dataset_file(dataset_files[0])

## Test local file

In [ ]:
import xarray as xr

In [ ]:
d = xr.open_dataset(local_file)

In [ ]:
d

In [ ]:
d.lead_time

In [ ]:
d.t2m.isel(lead_time=0, forecast_time=2, category=0).plot()

## Batch Download

In [ ]:
random.shuffle(dataset_files)

with multiprocessing.Pool(processes=16) as pool:
    for _ in tqdm(pool.imap(download_one_dataset_file, dataset_files), total=len(dataset_files)):
        pass        